In [1]:
import warnings
import pandas as pd
import numpy as np
import scipy.stats as st
from statsmodels.formula.api import ols
warnings.filterwarnings('ignore')

## Set11

In [122]:
df11 = pd.read_csv(path+'DataSet_11.csv')
df11.head()

,Country,Happiness_Rank,Happiness_Score,year
0,Switzerland,1,7.587,2015
1,Iceland,2,7.561,2015
2,Denmark,3,7.527,2015
3,Norway,4,7.522,2015
4,Canada,5,7.427,2015


In [123]:
# =============================================================================
# 1. O
# =============================================================================
q1 = pd.DataFrame(df11.Country.value_counts() < 3)
q1 = q1.reset_index()

print(len(q1[q1['Country']==True]))

q1.columns = ['Country', 'Counts']

df11 = pd.merge(df11, q1, on='Country', how='outer')
df11 = df11[df11['Counts']==False]

20


In [139]:
# =============================================================================
# 2. O 오랜만에 Df merge하는데 애먹었네
# =============================================================================

q2 = df11.copy()
tab = pd.pivot_table(q2, index='Country', columns='year', values='Happiness_Score')
tab['calc'] = (tab[2017] - tab[2015])/2
tab['calc'].sort_values(ascending=False)[:3]

Country
Latvia     0.3760
Romania    0.3505
Togo       0.3280
Name: calc, dtype: float64

In [144]:
# =============================================================================
# 3. X(첨배움)
# =============================================================================

# 세개집단. 아노바
from scipy.stats import f_oneway
from statsmodels.formula.api import ols

anova1 = f_oneway(tab[2015], tab[2016], tab[2017])
np.floor(anova1[0]*10000)/10000

0.0042

## Set12

In [145]:
df12 = pd.read_csv(path+'DataSet_12.csv')
df12.head()

,Age,Gender,Dependent_Count,Education_Level,is_Married,Read_Book_per_Year,Income_Range
0,52,M,2,석사,1,45,D
1,46,F,4,박사,1,26,X
2,53,F,0,석사,0,48,A
3,55,M,2,학사,1,45,D
4,51,F,3,석사,0,46,A


In [154]:
# =============================================================================
# 1. O .abs()를 꼭 사용할 것
# =============================================================================
q1 = df12.corr()[['Read_Book_per_Year']].drop('Read_Book_per_Year').abs().max()
print(round(q1, 3))

Read_Book_per_Year    0.797
dtype: float64


In [162]:
# =============================================================================
# 2.석사 이상(석사 및 박사) 여부에 따라서 연간 독서량 평균이 유의미하게 다른지 가설
# 검정을 활용하여 알아보고자 한다. 독립 2표본 t검정을 실시했을 때 
# 유의 확률(pvalue)의 값을 기술하시오.
# - 등분산 가정 하에서 검정을 실시한다.
# - 유의 확률은 반올림하여 소수점 셋째 자리까지 기술한다. (답안 예시) 0.123
# =============================================================================

q2 = df12[['Education_Level', 'Read_Book_per_Year']]
q2['Education_Level'] = q2['Education_Level'].replace('박사', '석사')
q2['Education_Level'] = q2['Education_Level'].replace('고졸', '학사')

A = q2[q2['Education_Level']=='학사'][['Read_Book_per_Year']]
B = q2[q2['Education_Level']=='석사'][['Read_Book_per_Year']]

from scipy.stats import ttest_ind

out = ttest_ind(A, B)
print(round(out[1][0], 3))

0.269


In [197]:
# =============================================================================
# 3. ㅁ, 10살 증가이므로 회귀계수에 10을 곱해준 값이 정답임!!
# =============================================================================
q3 = df12[(df12['Education_Level']!='고졸')&(df12['Income_Range'] != 'X')]
q3['Age'].astype('int32')

from statsmodels.formula.api import ols

var_lst = ['Age', 'Dependent_Count', 'is_Married']
form = 'Read_Book_per_Year~'+ '+'.join(var_lst)
out = ols(form, q3).fit()
print(f'{out.params[1]*10:.0f}')

8


## Set13

In [259]:
df13_train = pd.read_csv(path+'DataSet_13_train.csv')
df13_test = pd.read_csv(path+'DataSet_13_test.csv')
print(df13_train.shape, df13_test.shape)
df13_train.head()

(1500, 10) (500, 10)


,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target
0,0.516,Male,1,0,Graduate,STEM,13,5,36,1
1,0.766,Male,1,0,Graduate,STEM,12,5,264,0
2,0.920,Male,1,0,Masters,STEM,10,2,99,0
3,0.899,Male,1,0,Graduate,Business Degree,16,1,20,0
4,0.884,Male,1,0,Masters,STEM,20,5,39,0


In [213]:
# =============================================================================
# 1.(Dataset_13_train.csv를 활용하여) 경력과 최근 이직시 공백기간의 상관관계를 보고자
# 한다. 남여별 피어슨 상관계수를 각각 산출하고 더 높은 상관계수를 기술하시오.
# - 상관계수는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================
q1 = df13_train[['gender', 'experience', 'last_new_job']].dropna()

M = q1[q1['gender']=='Male'].corr()['experience'].drop('experience')
F = q1[q1['gender']=='Female'].corr()['experience'].drop('experience')

round(F,2)

last_new_job    0.45
Name: experience, dtype: float64

In [244]:
# =============================================================================
# 2. X 백분위수로 계산 안함. 답지랑 답이 다른데 과정은 같음 뭐지
# =============================================================================
# relevent_experience target 
# 카이제곱
base=q2['city_development_index'].quantile(0.85)
q2 = df13_train[(df13_train['major_discipline']=='STEM')|(df13_train['city_development_index']>base)]
q2['target'] = q2['target'].astype(str)

tab = pd.crosstab(index=q2['relevent_experience'], columns=q2['target'])

from scipy.stats import chi2_contingency

out = chi2_contingency(tab)
print(round(out[1],2))
out

0.0


(40.821089185161426,
 1.668230494942031e-10,
 1,
 array([[217.30663616,  73.69336384],
        [761.69336384, 258.30663616]]))

In [260]:
# =============================================================================
# 3. O
# =============================================================================
df13_train.dtypes != 'object'
df13_train = df13_train.drop(['gender', 'education_level', 'major_discipline'], axis=1)
df13_test = df13_test.drop(['gender', 'education_level', 'major_discipline'], axis=1)


from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=123).fit(df13_train.iloc[:, :-1], df13_train.iloc[:, -1])
res3 = dtc.score(df13_test.iloc[:, :-1], df13_test.iloc[:, -1])
round(res3, 2)

0.67

## Set14

In [261]:
df14 = pd.read_csv(path+'DataSet_14.csv')
df14.head()

,id,published,subject,level,price,subscribers,reviews,lectures,duration
0,1,2015-07-10,Business Finance,Beginner Level,1.7,56,8,18,2.000000
1,2,2016-10-25,Business Finance,All Levels,17.4,336,6,13,0.683333
2,3,2015-06-16,Business Finance,All Levels,8.3,871,79,59,2.000000
3,4,2016-07-28,Business Finance,All Levels,7.0,1095,17,35,3.000000
4,5,2016-05-18,Business Finance,Expert Level,8.3,913,117,48,2.000000


In [273]:
# =============================================================================
# 1. O
# =============================================================================
q1 = df14.copy()
q1['income'] = q1['price'] * df14['subscribers']
q1['rate'] = (q1['reviews'] / df14['subscribers'])
len(q1[(q1['income']>=10000)&(q1['rate']>=0.1)])

59

In [284]:
# =============================================================================
# 2. O
# =============================================================================
q2 = df14.copy()
q2['year'] = [i.split('-')[0] for i in q2['published']]

res2 = q2[(q2['year']=='2016')&(q2['subject']=='Web Development')][['price', 'subscribers']].corr()
round(res2['subscribers'].drop('subscribers'),2)

price    0.03
Name: subscribers, dtype: float64

In [304]:
# =============================================================================
# 3. X
#
# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm
# =============================================================================
q2['rate'] = (q2['reviews'] / q2['subscribers'])

from statsmodels.formula.api import ols

form= 'rate ~ C(year)' # 일원배치
out = ols(form, q2).fit()
round(out.fvalue,1)

18.5

## Set15

In [307]:
df_pos = pd.read_csv(path+'Dataset_05_Mart_POS.csv')
df_pos.head()

,Member_number,Date,itemDescription
0,1220,2015-01-01,canned beer
1,1220,2015-01-01,margarine
2,1220,2015-01-01,chocolate
3,1235,2015-01-01,sausage
4,1235,2015-01-01,sausage


In [306]:
df_item = pd.read_csv(path+'Dataset_05_item_list.csv')
df_item.head()

,prod_id,prod_nm,alcohol,frozen
0,1,tropical fruit,0,0
1,2,whole milk,0,0
2,3,pip fruit,0,0
3,4,other vegetables,0,0
4,5,rolls/buns,0,0


In [313]:
# =============================================================================
# 1.(Dataset_05_Mart_POS.csv를 활용하여) 가장 많은 제품이 팔린 날짜에 가장 많이 팔린
# 제품의 판매 개수는? (답안 예시) 1
# =============================================================================
date = df_pos['Date'].value_counts().idxmax()
q1 = df_pos[df_pos['Date']==date][['itemDescription']]
q1.value_counts().max()

7

In [380]:
# =============================================================================
# 2. X
# =============================================================================
# 가설: 금, 토가 주류 구매가 많음

pos2['month']=pd.to_datetime(pos2.Date).dt.month
pos2['day']=pd.to_datetime(pos2.Date).dt.day_name(locale='ko_kr')
pos2['week']=np.where(pos2['day'].isin(['금요일','토요일']), 1 , 0)

#      |  tm_wday
#      |      day of week, range [0, 6], Monday is 0



df_pos['day'] = [time.strptime(val, '%Y-%m-%d')[6] for val in df_pos['Date']]
df_pos['day'] = np.where(df_pos['day']==4, "1", 
                    (np.where(df_pos['day']==5,1,"0")))

q2=pd.merge(df_pos, df_item,
                left_on='itemDescription',
                right_on='prod_nm')
q2['month'] = pd.to_datetime(q2.Date).dt.month

q2 = q2[q2['month'] <= 3]

tab = pd.crosstab(index=q2.alcohol, columns=q2.day).dropna()
from scipy.stats import ttest_ind

out = ttest_ind(tab['1'], tab['0'])
# tab.columns
round(out[1],2)

0.59